* https://huggingface.co/docs/bitsandbytes/v0.43.2/index
* https://huggingface.co/docs/peft/en/index

In [1]:
!pip install -qU bitsandbytes datasets peft trl

In [4]:
import json
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["HF_HUB_DISABLE_PROGRESS_BARS"] = "1"

import datasets
from datasets import Dataset
import huggingface_hub
import peft
import torch
from torch import cuda as torch_cuda
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer
import trl

if False:
  !pwd
  import sys
  sys.path.append("../llm-learning")

from utils import jupyter_formatting

jupyter_formatting.setup_notebook_formatting()

torch_cuda.is_available()

True

In [5]:
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)

* https://huggingface.co/NousResearch/Meta-Llama-3.1-8B-Instruct

In [6]:
MODEL_ID = "NousResearch/Meta-Llama-3.1-8B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [7]:
if False:
    first_gpu = torch_cuda.get_device_properties(0)

    print(
        """\
        Memory allocated: {0:.2f} MB \
        Memory reserved: {1:.2f} MB\

        Total GPUs: {2}
        GPU's memory: {3:.1f} MB
        """.format(
                  torch_cuda.memory_allocated() / 1_024**2,
                  torch_cuda.memory_reserved() / 1_024**2,
                  torch_cuda.device_count(),
                  first_gpu.total_memory / 1_024**2,
              )
    )

In [8]:
if False:
    # Replace the model id with your specific model
    MODEL_ID = "NousResearch/Meta-Llama-3.1-8B-Instruct"

    # Download config.json file from the Hugging Face hub
    config_file_path = huggingface_hub.hf_hub_download(MODEL_ID, "config.json")
    print(config_file_path)

    index_file_path = huggingface_hub.hf_hub_download(
        MODEL_ID, "model.safetensors.index.json"
    )
    print(index_file_path)

In [9]:
if False:
    import json

    with open(config_file_path, "r") as config_file:
        config_data = json.load(config_file)
        print(json.dumps(config_data, indent=4))

    with open(index_file_path, "r") as index_file:
        index_data = json.load(index_file)
        print(json.dumps(index_data, indent=4))

In [10]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [11]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps

In [13]:
!git clone https://github.com/lauramanor/legal_summarization

In [14]:
jsonl_array = []
with open("legal_summarization/tldrlegal_v1.json") as f:
    data = json.load(f)
    for _, value in data.items():
        jsonl_array.append(value)

In [15]:
jsonl_array[0].keys()

dict_keys(['doc', 'id', 'original_text', 'reference_summary', 'title', 'uid'])

* https://huggingface.co/docs/datasets/en/index

In [16]:
legal_dataset = Dataset.from_list(jsonl_array)

In [17]:
legal_dataset

Dataset({
    features: ['doc', 'id', 'original_text', 'reference_summary', 'title', 'uid'],
    num_rows: 85
})

In [18]:
legal_dataset = legal_dataset.train_test_split(test_size=0.2)

In [19]:
legal_dataset_test_valid = legal_dataset["test"].train_test_split(test_size=0.5)

In [20]:
legal_dataset = datasets.DatasetDict(
    {
        "train": legal_dataset["train"],
        "test": legal_dataset_test_valid["test"],
        "validation": legal_dataset_test_valid["train"],
    }
)

In [21]:
legal_dataset

DatasetDict({
    train: Dataset({
        features: ['doc', 'id', 'original_text', 'reference_summary', 'title', 'uid'],
        num_rows: 68
    })
    test: Dataset({
        features: ['doc', 'id', 'original_text', 'reference_summary', 'title', 'uid'],
        num_rows: 9
    })
    validation: Dataset({
        features: ['doc', 'id', 'original_text', 'reference_summary', 'title', 'uid'],
        num_rows: 8
    })
})

In [22]:
legal_dataset["train"][0]

{
    "doc": "Pokemon GO Terms of Service",
    "id": "5786730a6cca83a54c0035b2",
    "original_text": "we may terminate your access to and use of the services at our sole
    discretion at any time and without notice to you. you may cancel your account at any time by
    accessing the pok\u00e9mon go help center available at https pokemongo nianticlabs com support
    delete en. upon any termination discontinuation or cancellation of services or your account the
    following provisions of these terms will survive arbitration notice content ownership rights
    granted by you effect of termination on trading items virtual money and virtual goods feedback
    disclaimer of warranties indemnity limitation of liability dispute resolution general terms and
    this sentence of termination.",
    "reference_summary": "both you and we can terminate your account and access to services at any
    time for any reason.",
    "title": "Termination",
    "uid": "legalsum13"
}

### Instruction Templating

https://www.llama.com/docs/model-cards-and-prompt-formats/llama3_1/

In [23]:
INSTRUCTION_PROMPT_TEMPLATE = """\
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Please convert the following legal content into a short human-readable summary<|eot_id|><|start_header_id|>user<|end_header_id|>

[LEGAL_DOC]{LEGAL_TEXT}[END_LEGAL_DOC]<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""

RESPONSE_TEMPLATE = """\
{NATURAL_LANGUAGE_SUMMARY}<|eot_id|><|end_of_text|>"""

In [24]:
def create_prompt(sample: str, include_response: bool = True) -> str:
    full_prompt = INSTRUCTION_PROMPT_TEMPLATE.format(LEGAL_TEXT=sample["original_text"])

    if include_response:
        full_prompt += RESPONSE_TEMPLATE.format(
            NATURAL_LANGUAGE_SUMMARY=sample["reference_summary"]
        )

    return full_prompt

In [25]:
print(create_prompt(legal_dataset["test"][1]))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Please convert the following legal content into a short human-readable summary<|eot_id|><|start_header_id|>user<|end_header_id|>

[LEGAL_DOC]you agree that if you use the sdk to develop applications you will protect the privacy and legal rights of users. if users provide you with user names passwords or other login information or personal information you must make the users aware that the information will be available to your application and you must provide legally adequate privacy notice and protection for those users. if your application stores personal or sensitive information provided by users it must do so securely. if users provide you with google account information your application may only use that information to access the user s google account when and for the limited purposes for which each user has given you permission to do so.[END_LEGAL_DOC]<|eot_id|><|start_header_id|>assistant<|end_header_id|>protect users s

In [26]:
def generate_response(prompt, model, tokenizer):
    """
    Parameters:
      - prompt: str representing formatted prompt
      - model: model object
      - tokenizer: tokenizer object

    Functionality:
      This will allow our model to generate a response to a prompt!

    Returns:
      - str response of the model
    """

    # convert str input into tokenized input
    encoded_input = tokenizer(prompt, return_tensors="pt")

    # send the tokenized inputs to our GPU
    model_inputs = encoded_input.to("cuda")

    # generate response and set desired generation parameters
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=256,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )

    # decode output from tokenized output to str output
    decoded_output = tokenizer.batch_decode(generated_ids)

    # return only the generated response (not the prompt) as output
    return decoded_output[0].split("<|end_header_id|>")[-1]

In [27]:
def generate_response(
    prompt: str,
    model: transformers.AutoModelForCausalLM,
    tokenizer: transformers.AutoTokenizer,
) -> str:
    # convert str input into tokenized input
    encoded_input = tokenizer(prompt, return_tensors="pt")

    # send the tokenized inputs to our GPU
    model_inputs = encoded_input.to("cuda")

    # generate response and set desired generation parameters
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=256,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )

    # decode output from tokenized output to str output
    decoded_output = tokenizer.batch_decode(generated_ids)

    # return only the generated response (not the prompt) as output
    return decoded_output[0].split("<|end_header_id|>")[-1]

In [28]:
generate_response(
    create_prompt(legal_dataset["test"][1], include_response=False), model, tokenizer
)

"\n\nHere is a short human-readable summary:\n\n**Protecting User Privacy and Rights**\n\nIf you use our SDK to develop an app, you must:\n\n* Respect users' privacy and legal rights\n* Clearly inform users when collecting their personal info (like names, passwords, or login details)\n* Provide adequate protection for users' personal info, especially if you store it securely\n* Only use Google account info with users' permission, and only for the specific purposes they've allowed.<|eot_id|>"

In [29]:
# Ground Truth Summary
legal_dataset["test"][1]["reference_summary"]

'protect users sensitive data and have an adequate privacy notice.'

In [30]:
# Let's try another just to see how the model responds to a different prompt.
generate_response(
    create_prompt(legal_dataset["test"][3], include_response=False), model, tokenizer
)

"\n\nHere is a short human-readable summary:\n\n**Community Guidelines**\n\nTo ensure a positive experience for everyone, please follow these guidelines when using our game:\n\n* Be respectful and considerate in your content.\n* Avoid sharing content that is:\n\t+ Racist or discriminatory\n\t+ Bullying or harassing\n\t+ Damaging to our reputation or others\n\t+ Pornographic or advertising someone else's content\n\t+ Impersonating a moderator or trying to trick or exploit others\n\nBy following these guidelines, you can help create a safe and enjoyable community for everyone.<|eot_id|>"

In [31]:
# Ground Truth Summary
legal_dataset["test"][3]["reference_summary"]

'things you make available on or through the game must not be offensive to people or illegal.'

### Required Post Processing

In [32]:
model_config = model.config
model = peft.prepare_model_for_kbit_training(model)

## Task #3: Setting up PEFT LoRA

In [33]:
def print_trainable_parameters(model: transformers.AutoModelForCausalLM) -> None:

    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()

    print(
        "trainable params: {0:,} || all params: {1:,} || trainable%: {2:,}".format(
            trainable_params, all_param, 100 * trainable_params / all_param
        )
    )

#### Initializing LoRA Config

https://huggingface.co/docs/peft/main/en/package_reference/lora#peft.LoraConfig
https://huggingface.co/docs/peft/main/en/package_reference/peft_model#peft.get_peft_model

In [34]:
# set our rank (higher value is more memory/better performance)
lora_r = 8

# set our dropout (default value)
lora_dropout = 0.1

# rule of thumb: alpha should be (lora_r * 2)
lora_alpha = 16

# construct our LoraConfig with the above hyperparameters
peft_config = peft.LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    target_modules="all-linear",
    task_type="CAUSAL_LM",
)

model = peft.get_peft_model(model, peft_config)

In [35]:
print_trainable_parameters(model)

trainable params: 20,971,520 || all params: 4,561,571,840 || trainable%: 0.4597432800707574


In [36]:
print(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lo

## Task #4: Training the Model


### Setting up Training

In [49]:
max_seq_length = 2_048

args = trl.SFTConfig(
    output_dir="llama381binstruct_summarize_short",
    # max_steps=500,
    max_steps=100,
    per_device_train_batch_size=1,
    warmup_steps=30,
    logging_steps=10,
    eval_strategy="steps",
    eval_steps=25,
    learning_rate=2e-4,
    lr_scheduler_type="linear",
    dataset_kwargs={
        "add_special_tokens": False,
        "append_concat_token": False,
    },
    max_seq_length=max_seq_length,
    packing=True,
)

* https://huggingface.co/docs/trl/en/sft_trainer

In [50]:
def _create_prompt_list(sample: str, include_response: bool = True) -> str:
    full_prompt = INSTRUCTION_PROMPT_TEMPLATE.format(LEGAL_TEXT=sample["original_text"])

    if include_response:
        full_prompt += RESPONSE_TEMPLATE.format(
            NATURAL_LANGUAGE_SUMMARY=sample["reference_summary"]
        )

    return [full_prompt]

In [51]:
args.packing = False
args.report_to = []

trainer = trl.SFTTrainer(
    model=model,
    peft_config=peft_config,
    tokenizer=tokenizer,
    formatting_func=_create_prompt_list,
    args=args,
    train_dataset=legal_dataset["train"],
    eval_dataset=legal_dataset["validation"],
)

Map:   0%|          | 0/68 [00:00<?, ? examples/s]

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [52]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss,Validation Loss
25,0.000500,3.816771
50,0.000100,3.955279
75,0.000100,3.949226
100,0.000000,3.940927


TrainOutput(global_step=100, training_loss=0.00010582007234916091, metrics={'train_runtime': 1045.8615, 'train_samples_per_second': 0.096, 'train_steps_per_second': 0.096, 'total_flos': 9247821240729600.0, 'train_loss': 0.00010582007234916091, 'epoch': 100.0})

## Task #5: Share Your Model!

In [ ]:
huggingface_hub.notebook_login()

In [ ]:
trainer.push_to_hub("SethWeidman/llama381binstruct_summarize_short_merged_test")

### Compare Outputs

In [ ]:
merged_model = model.merge_and_unload()

In [ ]:
merged_model.push_to_hub(
    "SethWeidman/llama381binstruct_summarize_short_merged_test",
    safe_serialization=True,
)

In [ ]:
tokenizer.push_to_hub("SethWeidman/llama381binstruct_summarize_short_merged_test")

In [ ]:
generate_response(
    create_prompt(legal_dataset["test"][1], include_response=False),
    merged_model,
    tokenizer,
)

In [ ]:
legal_dataset["test"][3]["original_text"]

In [ ]:
generate_response(
    create_prompt(legal_dataset["test"][3], include_response=False),
    merged_model,
    tokenizer,
)

In [ ]:
# Ground Truth Summary
legal_dataset["test"][3]["reference_summary"]